In [2]:
pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=450e01d5a490fcf850553a97651a03d065313b62e9d91ac2018cbcfda9e28998
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


In [3]:
import wikipedia

def get_article(search_term):
    results = wikipedia.search(search_term)
    first_result = results[0]
    page = wikipedia.page(first_result, auto_suggest=False)
    return page.content


In [4]:
article = get_article("Superman")
print(article[:500]) # article is very long, so let's just print a preview

Superman is a superhero created by writer Jerry Siegel and artist Joe Shuster, which first appeared in the comic book Action Comics #1, published in the United States on April 18, 1938. Superman has been regularly published in American comic books since 1938, and has been adapted to other media including radio serials, novels, films, television shows, theater, and video games.
Superman was born Kal-El, on the fictional planet Krypton. As a baby, his parents Jor-El and Lara sent him to Earth in a


Next up, we need to define our tool using the proper JSON Schema format. This is a very simple tool definition because the function expects a single argument: the search term string.

In [5]:
article_search_tool = {
    "name": "get_article",
    "description": "A tool to retrieve an up to date Wikipedia article.",
    "input_schema": {
        "type": "object",
        "properties": {
            "search_term": {
                "type": "string",
                "description": "The search term to find a wikipedia article by title"
            },
        },
        "required": ["search_term"]
    }
}

Next, we'll tell Claude it has access to the Wikipedia search tool and ask it to answer a question we know it cannot answer without the tool, like "Who won the 2024 Masters Tournament?"

In [6]:
pip install anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.3/289.3 kB 16.2 MB/s eta 0:00:00


In [7]:
from google.colab import userdata
CLAUDE_API_KEY = userdata.get('CLAUDE_API_KEY')

In [10]:
from anthropic import Anthropic
# from dotenv import load_dotenv

# load_dotenv()

client = Anthropic(api_key=CLAUDE_API_KEY)

messages = [{"role": "user", "content": "who won the 2024 Masters Tournament?"}]

response = client.messages.create(
    model="claude-sonnet-4-20250514",
    messages=messages,
    max_tokens=1000,
    tools=[article_search_tool]
)

In [11]:
response.content

[TextBlock(citations=None, text="I'll look up information about the 2024 Masters Tournament to find out who won.", type='text'),
 ToolUseBlock(id='toolu_011q9FMgGTwwsPxPA5vm1sF3', input={'search_term': '2024 Masters Tournament'}, name='get_article', type='tool_use')]

In [12]:
messages.append({"role": "assistant", "content": response.content})

In [13]:
messages

[{'role': 'user', 'content': 'who won the 2024 Masters Tournament?'},
 {'role': 'assistant',
  'content': [TextBlock(citations=None, text="I'll look up information about the 2024 Masters Tournament to find out who won.", type='text'),
   ToolUseBlock(id='toolu_011q9FMgGTwwsPxPA5vm1sF3', input={'search_term': '2024 Masters Tournament'}, name='get_article', type='tool_use')]}]

In [14]:
# This is a simple, but brittle way of getting the tool use information
# We're simply taking the last block from Claude's response.
tool_use = response.content[-1]
tool_name = tool_use.name
tool_input = tool_use.input
print("Tool name: ", tool_name)
print("Tool input", tool_input)

Tool name:  get_article
Tool input {'search_term': '2024 Masters Tournament'}


In [15]:
if tool_name == "get_article":
    search_term = tool_input["search_term"]
    wiki_result = get_article(search_term)
    print(f"Searching Wikipedia for {search_term}")
    print("WIKIPEDIA PAGE CONTENT:")
    print(wiki_result[:500]) #just printing a small bit of the article because it's so long

Searching Wikipedia for 2024 Masters Tournament
WIKIPEDIA PAGE CONTENT:
The 2024 Masters Tournament was the 88th edition of the Masters Tournament and the first of the men's four major golf championships held in 2024. The tournament was played from April 11–14 at Augusta National Golf Club in Augusta, Georgia, United States. Scottie Scheffler won his second Masters and major, four strokes ahead of runner-up Ludvig Åberg, who was playing in his first major. Scheffler became the fourth-youngest two-time winner of the tournament and the second player, after Tiger Woods


In [16]:
tool_response = {
    "role": "user",
    "content": [
        {
        "type": "tool_result",
        "tool_use_id": tool_use.id,
        "content": wiki_result
        }
    ]
}

In [17]:
tool_response

{'role': 'user',
 'content': [{'type': 'tool_result',
   'tool_use_id': 'toolu_011q9FMgGTwwsPxPA5vm1sF3',
   'content': "The 2024 Masters Tournament was the 88th edition of the Masters Tournament and the first of the men's four major golf championships held in 2024. The tournament was played from April 11–14 at Augusta National Golf Club in Augusta, Georgia, United States. Scottie Scheffler won his second Masters and major, four strokes ahead of runner-up Ludvig Åberg, who was playing in his first major. Scheffler became the fourth-youngest two-time winner of the tournament and the second player, after Tiger Woods in 2001, to win both the Masters and The Players Championship in the same calendar year. \n\n\n== Course ==\n\nThe only change to the course for the 2024 tournament was the lengthening of the second hole by ten yards.\n\n\n== Field ==\nParticipation in the Masters Tournament is by invitation only, and the tournament has the smallest field of the major championships. There are

In [18]:
messages.append(tool_response)

In [19]:
messages

[{'role': 'user', 'content': 'who won the 2024 Masters Tournament?'},
 {'role': 'assistant',
  'content': [TextBlock(citations=None, text="I'll look up information about the 2024 Masters Tournament to find out who won.", type='text'),
   ToolUseBlock(id='toolu_011q9FMgGTwwsPxPA5vm1sF3', input={'search_term': '2024 Masters Tournament'}, name='get_article', type='tool_use')]},
 {'role': 'user',
  'content': [{'type': 'tool_result',
    'tool_use_id': 'toolu_011q9FMgGTwwsPxPA5vm1sF3',
    'content': "The 2024 Masters Tournament was the 88th edition of the Masters Tournament and the first of the men's four major golf championships held in 2024. The tournament was played from April 11–14 at Augusta National Golf Club in Augusta, Georgia, United States. Scottie Scheffler won his second Masters and major, four strokes ahead of runner-up Ludvig Åberg, who was playing in his first major. Scheffler became the fourth-youngest two-time winner of the tournament and the second player, after Tiger Wo

In [20]:
follow_up_response = client.messages.create(
    model="claude-3-sonnet-20240229",
    messages=messages,
    max_tokens=1000,
    tools=[article_search_tool]
)

/tmp/ipython-input-20-267033875.py:1: DeprecationWarning: The model 'claude-3-sonnet-20240229' is deprecated and will reach end-of-life on July 21st, 2025.
Please migrate to a newer model. Visit https://docs.anthropic.com/en/docs/resources/model-deprecations for more information.
  follow_up_response = client.messages.create(


In [21]:
follow_up_response.content[0].text

'Based on the Wikipedia article, Scottie Scheffler won the 2024 Masters Tournament, his second Masters title. He shot a final round 68 to win by 4 strokes over runner-up Ludvig Åberg, who was making his Masters debut. Scheffler became the fourth-youngest two-time Masters winner, and also won The Players Championship earlier in 2024, becoming just the second player after Tiger Woods in 2001 to win both events in the same year.'

In [26]:
"""
Module for answering questions using Claude API with article search tool.

This module provides functionality to interact with Claude's API to answer questions,
with the ability to search for and retrieve Wikipedia articles when needed.
"""

from typing import List, Dict, Any, Optional


def answer_question(question: str) -> None:
    """
    Answer a question using Claude API with optional article search tool.

    This function sends a question to Claude's API and handles tool use requests
    for article searches. If Claude requests to search for an article, the function
    retrieves the Wikipedia content and provides it back to Claude for a final answer.

    Args:
        question (str): The question to be answered by Claude.

    Returns:
        None: The function prints the response directly to stdout.

    Raises:
        AttributeError: If the response object doesn't have expected attributes.
        KeyError: If expected keys are missing from the tool input.
        IndexError: If the response content list is empty.
    """
    # Initialize the conversation with the user's question
    messages: List[Dict[str, Any]] = [{"role": "user", "content": question}]

    # Send the initial request to Claude with tool availability
    response = client.messages.create(
        model="claude-sonnet-4-20250514",
        messages=messages,
        max_tokens=1000,
        tools=[article_search_tool]
    )

    # Check if Claude wants to use the article search tool
    if response.stop_reason == "tool_use":
        # Extract tool use information from Claude's response
        tool_use = response.content[-1]  # Get the last content item (tool use)
        tool_name: str = tool_use.name
        tool_input: Dict[str, Any] = tool_use.input

        # Add Claude's tool use request to the conversation history
        messages.append({"role": "assistant", "content": response.content})

        # Handle the article search tool specifically
        if tool_name == "get_article":
            # Extract the search term from the tool input
            search_term: str = tool_input["search_term"]
            print(f"Claude wants to get an article for {search_term}")

            # Retrieve the Wikipedia article content
            wiki_result: str = get_article(search_term)

            # Construct the tool result message to send back to Claude
            tool_response: Dict[str, Any] = {
                "role": "user",
                "content": [
                    {
                        "type": "tool_result",
                        "tool_use_id": tool_use.id,  # Match the tool use ID
                        "content": wiki_result
                    }
                ]
            }

            # Add the tool result to the conversation history
            messages.append(tool_response)

            # Send the tool result back to Claude for final processing
            response = client.messages.create(
                model="claude-3-sonnet-20240229",
                messages=messages,
                max_tokens=1000,
                tools=[article_search_tool]
            )

            # Display Claude's final answer after processing the article
            print("Claude's final answer:")
            print(response.content[0].text)

    else:
        # Claude answered directly without needing to search for articles
        print("Claude did not call our tool")
        print(response.content[0].text)

In [27]:
answer_question("Who won the 2024 F1 Australian Grand Prix")

Claude wants to get an article for 2024 Australian Grand Prix


/tmp/ipython-input-26-2926702251.py:76: DeprecationWarning: The model 'claude-3-sonnet-20240229' is deprecated and will reach end-of-life on July 21st, 2025.
Please migrate to a newer model. Visit https://docs.anthropic.com/en/docs/resources/model-deprecations for more information.
  response = client.messages.create(


Claude's final answer:
Based on the Wikipedia article summary, Carlos Sainz Jr. of Ferrari won the 2024 Australian Grand Prix, with his teammate Charles Leclerc finishing second. It was Sainz's first win of the 2024 season after undergoing an appendectomy prior to the previous race in Saudi Arabia. Lando Norris secured his 14th career podium with a 3rd place finish, surpassing Nick Heidfeld's record for most podiums without a win. The race saw a high attrition rate, with Max Verstappen retiring early due to brake failure after starting on pole, ending his streak of 9 consecutive wins. The event set a new attendance record at Albert Park.


In [28]:
"""
Module for managing conversations with Claude API using tool-enabled chat sessions.

This module provides a class-based interface for interacting with Claude's API,
maintaining conversation history, and handling tool use requests dynamically.
"""

from typing import List, Dict, Any, Optional, Callable
from abc import ABC, abstractmethod


class ToolHandler(ABC):
    """Abstract base class for tool handlers."""

    @abstractmethod
    def handle(self, tool_input: Dict[str, Any]) -> str:
        """
        Handle the tool execution.

        Args:
            tool_input: The input parameters for the tool.

        Returns:
            str: The result of the tool execution.
        """


class ArticleSearchHandler(ToolHandler):
    """Handler for article search tool."""

    def __init__(self, get_article_func: Callable[[str], str]):
        """
        Initialize the article search handler.

        Args:
            get_article_func: Function to retrieve articles by search term.
        """
        self.get_article_func = get_article_func

    def handle(self, tool_input: Dict[str, Any]) -> str:
        """
        Handle article search requests.

        Args:
            tool_input: Dictionary containing 'search_term' key.

        Returns:
            str: The retrieved article content.

        Raises:
            KeyError: If 'search_term' is missing from tool_input.
        """
        search_term = tool_input["search_term"]
        return self.get_article_func(search_term)


class ClaudeChatManager:
    """
    Manager class for handling conversations with Claude API using tools.

    This class maintains conversation history, handles tool use requests,
    and provides a clean interface for interacting with Claude's API.
    """

    def __init__(
        self,
        client: Any,
        model: str = "claude-sonnet-4-20250514",
        max_tokens: int = 1000,
        tools: Optional[List[Dict[str, Any]]] = None
    ):
        """
        Initialize the Claude chat manager.

        Args:
            client: The Claude API client instance.
            model: The Claude model to use for conversations.
            max_tokens: Maximum tokens for each API response.
            tools: List of tool definitions available to Claude.
        """
        self.client = client
        self.model = model
        self.max_tokens = max_tokens
        self.tools = tools or []
        self.messages: List[Dict[str, Any]] = []
        self.tool_handlers: Dict[str, ToolHandler] = {}

    def register_tool_handler(self, tool_name: str, handler: ToolHandler) -> None:
        """
        Register a handler for a specific tool.

        Args:
            tool_name: The name of the tool to handle.
            handler: The ToolHandler instance to process this tool.
        """
        self.tool_handlers[tool_name] = handler

    def get_history(self) -> List[Dict[str, Any]]:
        """
        Get the complete conversation history.

        Returns:
            List[Dict[str, Any]]: A copy of the current conversation history.
        """
        return self.messages.copy()

    def append_to_history(self, message: Dict[str, Any]) -> None:
        """
        Append a message to the conversation history.

        Args:
            message: The message dictionary to add to history.

        Raises:
            ValueError: If the message doesn't have required 'role' key.
        """
        if "role" not in message:
            raise ValueError("Message must contain 'role' key")

        self.messages.append(message)

    def clear_history(self) -> None:
        """Clear the entire conversation history."""
        self.messages.clear()

    def _handle_tool_use(self, response: Any) -> Any:
        """
        Handle tool use requests from Claude's response.

        Args:
            response: The response object from Claude API.

        Returns:
            Any: The final response after tool processing.

        Raises:
            ValueError: If tool handler is not registered for the requested tool.
            AttributeError: If response object doesn't have expected attributes.
            IndexError: If response content is empty.
        """
        # Extract tool use information from Claude's response
        tool_use = response.content[-1]  # Get the last content item (tool use)
        tool_name: str = tool_use.name
        tool_input: Dict[str, Any] = tool_use.input

        print(f"Claude wants to use tool: {tool_name}")

        # Add Claude's tool use request to the conversation history
        self.append_to_history({"role": "assistant", "content": response.content})

        # Check if we have a handler for this tool
        if tool_name not in self.tool_handlers:
            raise ValueError(f"No handler registered for tool: {tool_name}")

        # Execute the tool using the registered handler
        tool_result: str = self.tool_handlers[tool_name].handle(tool_input)

        # Construct the tool result message to send back to Claude
        tool_response: Dict[str, Any] = {
            "role": "user",
            "content": [
                {
                    "type": "tool_result",
                    "tool_use_id": tool_use.id,  # Match the tool use ID
                    "content": tool_result
                }
            ]
        }

        # Add the tool result to the conversation history
        self.append_to_history(tool_response)

        # Send the tool result back to Claude for final processing
        final_response = self.client.messages.create(
            model=self.model,
            messages=self.messages,
            max_tokens=self.max_tokens,
            tools=self.tools
        )

        return final_response

    def invoke_chat(self, question: str, print_response: bool = True) -> str:
        """
        Send a question to Claude and handle any tool use requests.

        Args:
            question: The question or message to send to Claude.
            print_response: Whether to print the response to stdout.

        Returns:
            str: Claude's final response text.

        Raises:
            AttributeError: If the response object doesn't have expected attributes.
            IndexError: If the response content list is empty.
            ValueError: If a requested tool handler is not registered.
        """
        # Add the user's question to conversation history
        user_message = {"role": "user", "content": question}
        self.append_to_history(user_message)

        # Send the request to Claude with tool availability
        response = self.client.messages.create(
            model=self.model,
            messages=self.messages,
            max_tokens=self.max_tokens,
            tools=self.tools
        )

        # Handle tool use if Claude requested it
        if response.stop_reason == "tool_use":
            response = self._handle_tool_use(response)
        else:
            # Claude answered directly without using tools
            if print_response:
                print("Claude answered directly without using tools")

        # Extract the final response text
        final_text = response.content[0].text

        # Add Claude's final response to history
        self.append_to_history({"role": "assistant", "content": response.content})

        if print_response:
            print("Claude's response:")
            print(final_text)

        return final_text

    def add_system_message(self, system_content: str) -> None:
        """
        Add a system message to the conversation history.

        Args:
            system_content: The system message content.
        """
        system_message = {"role": "system", "content": system_content}
        # Insert system message at the beginning if messages exist
        if self.messages:
            self.messages.insert(0, system_message)
        else:
            self.append_to_history(system_message)


# Example usage function to demonstrate the class
def create_chat_manager_example(client: Any, get_article_func: Callable[[str], str]) -> ClaudeChatManager:
    """
    Create a configured ClaudeChatManager instance with article search capability.

    Args:
        client: The Claude API client instance.
        get_article_func: Function to retrieve articles by search term.

    Returns:
        ClaudeChatManager: Configured chat manager with article search tool.
    """
    # Define the article search tool (you'll need to provide this from your original code)
    article_search_tool = {
        "name": "get_article",
        "description": "Search for and retrieve Wikipedia articles",
        "input_schema": {
            "type": "object",
            "properties": {
                "search_term": {
                    "type": "string",
                    "description": "The term to search for in Wikipedia"
                }
            },
            "required": ["search_term"]
        }
    }

    # Create the chat manager
    chat_manager = ClaudeChatManager(
        client=client,
        tools=[article_search_tool]
    )

    # Register the article search handler
    article_handler = ArticleSearchHandler(get_article_func)
    chat_manager.register_tool_handler("get_article", article_handler)

    return chat_manager

In [29]:
# Create the chat manager
chat_manager = create_chat_manager_example(client, get_article)

# Have a conversation
response = chat_manager.invoke_chat("What is quantum computing?")

# Get conversation history
history = chat_manager.get_history()

# Add system instructions
chat_manager.add_system_message("Be concise in your responses")

Claude wants to use tool: get_article
Claude's response:
Based on the Wikipedia article, I can provide you with a comprehensive explanation of quantum computing:

## What is Quantum Computing?

Quantum computing is a revolutionary computational paradigm that exploits quantum mechanical phenomena to process information in fundamentally different ways than classical computers. Instead of using traditional bits that can only be in states of 0 or 1, quantum computers use **qubits** (quantum bits) that can exist in a **superposition** of both states simultaneously.

## Key Principles

### 1. **Qubits and Superposition**
- While classical bits are either 0 or 1, qubits can exist in a combination of both states at once
- This is represented mathematically as α|0⟩ + β|1⟩, where α and β are probability amplitudes
- This allows quantum computers to process multiple possibilities simultaneously

### 2. **Quantum Entanglement**
- Qubits can be "entangled," meaning they become correlated in ways th

In [30]:
"""
Module for managing conversations with Claude API using multiple tools.

This module provides a class-based interface for interacting with Claude's API,
maintaining conversation history, and handling multiple tool use requests dynamically.
Supports article search and web search tools with extensible architecture.
"""

from typing import List, Dict, Any, Optional, Callable
from abc import ABC, abstractmethod


class ToolHandler(ABC):
    """Abstract base class for tool handlers."""

    @abstractmethod
    def handle(self, tool_input: Dict[str, Any]) -> str:
        """
        Handle the tool execution.

        Args:
            tool_input: The input parameters for the tool.

        Returns:
            str: The result of the tool execution.
        """


class ArticleSearchHandler(ToolHandler):
    """Handler for article search tool."""

    def __init__(self, get_article_func: Callable[[str], str]):
        """
        Initialize the article search handler.

        Args:
            get_article_func: Function to retrieve articles by search term.
        """
        self.get_article_func = get_article_func

    def handle(self, tool_input: Dict[str, Any]) -> str:
        """
        Handle article search requests.

        Args:
            tool_input: Dictionary containing 'search_term' key.

        Returns:
            str: The retrieved article content.

        Raises:
            KeyError: If 'search_term' is missing from tool_input.
        """
        search_term = tool_input["search_term"]
        print(f"Searching for article: {search_term}")
        return self.get_article_func(search_term)


class WebSearchHandler(ToolHandler):
    """Handler for web search tool."""

    def __init__(self, web_search_func: Callable[[str], str]):
        """
        Initialize the web search handler.

        Args:
            web_search_func: Function to perform web searches by topic.
        """
        self.web_search_func = web_search_func

    def handle(self, tool_input: Dict[str, Any]) -> str:
        """
        Handle web search requests.

        Args:
            tool_input: Dictionary containing 'topic' key.

        Returns:
            str: The web search results.

        Raises:
            KeyError: If 'topic' is missing from tool_input.
        """
        topic = tool_input["topic"]
        print(f"Searching the web for: {topic}")
        return self.web_search_func(topic)


class MultiToolChatManager:
    """
    Manager class for handling conversations with Claude API using multiple tools.

    This class maintains conversation history, handles multiple tool use requests,
    and provides a clean interface for interacting with Claude's API with support
    for article search, web search, and other extensible tools.
    """

    def __init__(
        self,
        client: Any,
        model: str = "claude-sonnet-4-20250514",
        max_tokens: int = 1000,
        tools: Optional[List[Dict[str, Any]]] = None
    ):
        """
        Initialize the multi-tool chat manager.

        Args:
            client: The Claude API client instance.
            model: The Claude model to use for conversations.
            max_tokens: Maximum tokens for each API response.
            tools: List of tool definitions available to Claude.
        """
        self.client = client
        self.model = model
        self.max_tokens = max_tokens
        self.tools = tools or []
        self.messages: List[Dict[str, Any]] = []
        self.tool_handlers: Dict[str, ToolHandler] = {}
        self._tool_call_count = 0  # Track number of tool calls for debugging

    def register_tool_handler(self, tool_name: str, handler: ToolHandler) -> None:
        """
        Register a handler for a specific tool.

        Args:
            tool_name: The name of the tool to handle.
            handler: The ToolHandler instance to process this tool.
        """
        self.tool_handlers[tool_name] = handler
        print(f"Registered handler for tool: {tool_name}")

    def get_registered_tools(self) -> List[str]:
        """
        Get list of all registered tool names.

        Returns:
            List[str]: Names of all registered tools.
        """
        return list(self.tool_handlers.keys())

    def get_history(self) -> List[Dict[str, Any]]:
        """
        Get the complete conversation history.

        Returns:
            List[Dict[str, Any]]: A copy of the current conversation history.
        """
        return self.messages.copy()

    def append_to_history(self, message: Dict[str, Any]) -> None:
        """
        Append a message to the conversation history.

        Args:
            message: The message dictionary to add to history.

        Raises:
            ValueError: If the message doesn't have required 'role' key.
        """
        if "role" not in message:
            raise ValueError("Message must contain 'role' key")

        self.messages.append(message)

    def clear_history(self) -> None:
        """Clear the entire conversation history and reset tool call counter."""
        self.messages.clear()
        self._tool_call_count = 0
        print("Conversation history cleared")

    def get_tool_call_count(self) -> int:
        """
        Get the number of tool calls made in the current conversation.

        Returns:
            int: Number of tool calls made.
        """
        return self._tool_call_count

    def _handle_tool_use(self, response: Any) -> Any:
        """
        Handle tool use requests from Claude's response.

        Args:
            response: The response object from Claude API.

        Returns:
            Any: The final response after tool processing.

        Raises:
            ValueError: If tool handler is not registered for the requested tool.
            AttributeError: If response object doesn't have expected attributes.
            IndexError: If response content is empty.
        """
        # Extract tool use information from Claude's response
        tool_use = response.content[-1]  # Get the last content item (tool use)
        tool_name: str = tool_use.name
        tool_input: Dict[str, Any] = tool_use.input

        # Increment tool call counter
        self._tool_call_count += 1

        print(f"Claude wants to use tool: {tool_name} (Call #{self._tool_call_count})")

        # Add Claude's tool use request to the conversation history
        self.append_to_history({"role": "assistant", "content": response.content})

        # Check if we have a handler for this tool
        if tool_name not in self.tool_handlers:
            available_tools = ", ".join(self.get_registered_tools())
            raise ValueError(
                f"No handler registered for tool: {tool_name}. "
                f"Available tools: {available_tools}"
            )

        # Execute the tool using the registered handler
        try:
            tool_result: str = self.tool_handlers[tool_name].handle(tool_input)
            print(f"Tool {tool_name} executed successfully")
        except Exception as e:
            # Handle tool execution errors gracefully
            tool_result = f"Error executing tool {tool_name}: {str(e)}"
            print(f"Error in tool {tool_name}: {str(e)}")

        # Construct the tool result message to send back to Claude
        tool_response: Dict[str, Any] = {
            "role": "user",
            "content": [
                {
                    "type": "tool_result",
                    "tool_use_id": tool_use.id,  # Match the tool use ID
                    "content": tool_result
                }
            ]
        }

        # Add the tool result to the conversation history
        self.append_to_history(tool_response)

        # Send the tool result back to Claude for final processing
        final_response = self.client.messages.create(
            model=self.model,
            messages=self.messages,
            max_tokens=self.max_tokens,
            tools=self.tools
        )

        return final_response

    def invoke_chat(self, question: str, print_response: bool = True) -> str:
        """
        Send a question to Claude and handle any tool use requests.

        This method supports recursive tool calls - Claude can use multiple tools
        in sequence to answer a single question.

        Args:
            question: The question or message to send to Claude.
            print_response: Whether to print the response to stdout.

        Returns:
            str: Claude's final response text.

        Raises:
            AttributeError: If the response object doesn't have expected attributes.
            IndexError: If the response content list is empty.
            ValueError: If a requested tool handler is not registered.
        """
        # Add the user's question to conversation history
        user_message = {"role": "user", "content": question}
        self.append_to_history(user_message)

        # Send the request to Claude with tool availability
        response = self.client.messages.create(
            model=self.model,
            messages=self.messages,
            max_tokens=self.max_tokens,
            tools=self.tools
        )

        # Handle tool use if Claude requested it (supports multiple tool calls)
        max_tool_iterations = 5  # Prevent infinite tool calling loops
        iteration_count = 0

        while response.stop_reason == "tool_use" and iteration_count < max_tool_iterations:
            response = self._handle_tool_use(response)
            iteration_count += 1

            if iteration_count >= max_tool_iterations:
                print(f"Warning: Reached maximum tool iterations ({max_tool_iterations})")
                break

        if response.stop_reason != "tool_use":
            # Claude provided a final answer
            if print_response and iteration_count == 0:
                print("Claude answered directly without using tools")
            elif print_response and iteration_count > 0:
                print(f"Claude completed answer after {iteration_count} tool call(s)")

        # Extract the final response text
        final_text = response.content[0].text

        # Add Claude's final response to history
        self.append_to_history({"role": "assistant", "content": response.content})

        if print_response:
            print("Claude's response:")
            print(final_text)

        return final_text

    def add_system_message(self, system_content: str) -> None:
        """
        Add a system message to the conversation history.

        Args:
            system_content: The system message content.
        """
        system_message = {"role": "system", "content": system_content}
        # Insert system message at the beginning if messages exist
        if self.messages:
            self.messages.insert(0, system_message)
        else:
            self.append_to_history(system_message)
        print("System message added to conversation")

    def remove_tool(self, tool_name: str) -> bool:
        """
        Remove a tool handler and its definition.

        Args:
            tool_name: The name of the tool to remove.

        Returns:
            bool: True if tool was removed, False if tool was not found.
        """
        if tool_name in self.tool_handlers:
            del self.tool_handlers[tool_name]
            # Remove from tools list
            self.tools = [tool for tool in self.tools if tool.get("name") != tool_name]
            print(f"Removed tool: {tool_name}")
            return True
        return False

    def add_tool_definition(self, tool_definition: Dict[str, Any]) -> None:
        """
        Add a new tool definition to the available tools.

        Args:
            tool_definition: The tool definition dictionary.

        Raises:
            ValueError: If tool definition is missing required fields.
        """
        required_fields = ["name", "description", "input_schema"]
        for field in required_fields:
            if field not in tool_definition:
                raise ValueError(f"Tool definition missing required field: {field}")

        # Check if tool already exists and replace it
        tool_name = tool_definition["name"]
        self.tools = [tool for tool in self.tools if tool.get("name") != tool_name]
        self.tools.append(tool_definition)
        print(f"Added tool definition: {tool_name}")


# Predefined tool definitions
ARTICLE_SEARCH_TOOL = {
    "name": "get_article",
    "description": "Search for and retrieve Wikipedia articles",
    "input_schema": {
        "type": "object",
        "properties": {
            "search_term": {
                "type": "string",
                "description": "The term to search for in Wikipedia"
            }
        },
        "required": ["search_term"]
    }
}

WEB_SEARCH_TOOL = {
    "name": "web_search",
    "description": "A tool to retrieve up to date information on a given topic by searching the web",
    "input_schema": {
        "type": "object",
        "properties": {
            "topic": {
                "type": "string",
                "description": "The topic to search the web for"
            },
        },
        "required": ["topic"]
    }
}


def create_multi_tool_chat_manager(
    client: Any,
    get_article_func: Optional[Callable[[str], str]] = None,
    web_search_func: Optional[Callable[[str], str]] = None,
    include_article_search: bool = True,
    include_web_search: bool = True
) -> MultiToolChatManager:
    """
    Create a configured MultiToolChatManager with multiple tools.

    Args:
        client: The Claude API client instance.
        get_article_func: Function to retrieve articles by search term.
        web_search_func: Function to perform web searches by topic.
        include_article_search: Whether to include article search tool.
        include_web_search: Whether to include web search tool.

    Returns:
        MultiToolChatManager: Configured chat manager with requested tools.

    Raises:
        ValueError: If a tool is requested but its function is not provided.
    """
    # Prepare tools list
    tools = []

    if include_article_search:
        if get_article_func is None:
            raise ValueError("get_article_func must be provided when include_article_search=True")
        tools.append(ARTICLE_SEARCH_TOOL)

    if include_web_search:
        if web_search_func is None:
            raise ValueError("web_search_func must be provided when include_web_search=True")
        tools.append(WEB_SEARCH_TOOL)

    # Create the chat manager
    chat_manager = MultiToolChatManager(
        client=client,
        tools=tools
    )

    # Register tool handlers
    if include_article_search and get_article_func:
        article_handler = ArticleSearchHandler(get_article_func)
        chat_manager.register_tool_handler("get_article", article_handler)

    if include_web_search and web_search_func:
        web_handler = WebSearchHandler(web_search_func)
        chat_manager.register_tool_handler("web_search", web_handler)

    print(f"Multi-tool chat manager created with {len(tools)} tool(s)")
    return chat_manager


def create_article_only_chat_manager(
    client: Any,
    get_article_func: Callable[[str], str]
) -> MultiToolChatManager:
    """
    Create a chat manager with only article search capability.

    Args:
        client: The Claude API client instance.
        get_article_func: Function to retrieve articles by search term.

    Returns:
        MultiToolChatManager: Chat manager with article search tool only.
    """
    return create_multi_tool_chat_manager(
        client=client,
        get_article_func=get_article_func,
        include_web_search=False
    )


def create_web_only_chat_manager(
    client: Any,
    web_search_func: Callable[[str], str]
) -> MultiToolChatManager:
    """
    Create a chat manager with only web search capability.

    Args:
        client: The Claude API client instance.
        web_search_func: Function to perform web searches by topic.

    Returns:
        MultiToolChatManager: Chat manager with web search tool only.
    """
    return create_multi_tool_chat_manager(
        client=client,
        web_search_func=web_search_func,
        include_article_search=False
    )

Define custom tool: `get_article` a function to search items from wikipedia

In [35]:
import wikipedia

def get_article(search_term):
    results = wikipedia.search(search_term)
    first_result = results[0]
    page = wikipedia.page(first_result, auto_suggest=False)
    return page.content

In [36]:
pip install serpapi google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=20e8996851d5b8a2b7daab8753b3c4ab2a35342d6b3e162aafede1f77b1a7c18
  Stored in directory: /root/.cache/pip/wheels/6e/42/3e/aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results


Define custom tool: `search_serpapi` a function to search the internet

In [37]:
from google.colab import userdata

In [47]:
from serpapi import GoogleSearch
from typing import List, Dict, Any

def search_serpapi(query: str) -> List[Dict[str, Any]]:
    """
    Search using SerpAPI for the given query and return the results.

    :param query: The search query string.
    :return: A list of search results.
    :raises Exception: For any errors during the request.
    """
    SERPAPI_API_KEY = userdata.get('SERPAPI_API_KEY')
    try:
        search = GoogleSearch({
            "q": query,
            "location": "Austin, Texas, United States",
            "api_key": SERPAPI_API_KEY
        })
        raw_results = search.get_dict()
        results = raw_results.get("organic_results", [])
        snippet = ' '.join([results[i]['snippet'] for i in range(len(results))])
        return snippet
    except Exception as e:
        raise Exception(f"An error occurred: {e}")

In [48]:
search_serpapi("tesla")

'Model X All-Wheel Drive starts at $91,630. Price includes Destination and Order Fees but exclude taxes and other fees. Subject to change. Vehicle shown has ... Tesla, Inc is an American multinational automotive and clean energy company. Headquartered in Austin, Texas, it designs, manufactures and sells battery ... Tesla, Inc. designs, develops, manufactures, leases, and sells electric vehicles, and energy generation and storage systems in the United States, China, ...'

In [49]:
# Create manager with both tools
chat_manager = create_multi_tool_chat_manager(
    client=client,
    get_article_func=get_article,
    web_search_func=search_serpapi
)

# Use the manager
response = chat_manager.invoke_chat("What's the latest news about Tesla?")

# Check what tools were used
print(f"Tools used: {chat_manager.get_tool_call_count()}")
print(f"Available tools: {chat_manager.get_registered_tools()}")

Registered handler for tool: get_article
Registered handler for tool: web_search
Multi-tool chat manager created with 2 tool(s)
Claude wants to use tool: web_search (Call #1)
Searching the web for: Tesla latest news
Tool web_search executed successfully
Claude completed answer after 1 tool call(s)
Claude's response:
Based on the latest search results, here are some key recent developments with Tesla:

## Recent Tesla News Highlights:

**New Product Launches:**
- Tesla is launching a new **Model 3 Performance** - described as a highly differentiated performance trim using Tesla's latest manufacturing and engineering capabilities

**Autonomous Driving Progress:**
- A Tesla vehicle reportedly **delivered itself to a customer autonomously**, as confirmed by Elon Musk
- Tesla's **Robotaxi program** is sending out more invites for next phases, though first impressions have been mixed due to some operational mistakes
- The company has hired **Henry Kuang**, former autonomy lead at GM's Cruise